In [ ]:
from PIL import Image, ImageDraw
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2
from collections.abc import Sequence

In [ ]:

def draw_rgb_predictions_cv2(image, predictions, confidence_threshold=0.75):
    print(f"RGB encontradas: {len(predictions)}")
    for pred in predictions:
        bbox = list(map(int, pred["bbox"]))
        class_id = pred["class_id"]
        score = pred["confidence"]

        if score < confidence_threshold:
            continue

        if class_id == 1:
            label = f"person: {score:.2f}"
            color = (0, 165, 255)  # orange (BGR)
        elif class_id == 3:
            label = f"car: {score:.2f}"
            color = (0, 255, 0)  # green (BGR)
        else:
            continue

        x1, y1, x2, y2 = bbox
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, max(0, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def draw_event_predictions_cv2(image, predictions, confidence_threshold=0.75,y_shift=0):
    print(f"Event encontradas: {len(predictions)}")
    for pred in predictions:
        x = int(pred["x"])
        y = max(int(pred["y"]) - y_shift, 0)
        w = int(pred["w"])
        h = int(pred["h"])
        class_id = pred["class_id"]
        score = pred["class_confidence"]

        if score < confidence_threshold:
            continue

        if class_id == 0:
            label = f"person_evt: {score:.2f}"
            color = (255, 0, 0)  # blue
        elif class_id == 2:
            label = f"car_evt: {score:.2f}"
            color = (128, 0, 128)  # purple
        else:
            continue

        x2, y2 = x + w, y + h
        cv2.rectangle(image, (x, y), (x2, y2), color, 2)
        cv2.putText(image, label, (x, max(0, y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def draw_ground_truth_cv2(image, annotations,y_shift):
    print(f"GT encontradas: {len(annotations)}")
    for gt in annotations:
        x = int(gt["x"])
        y = max(int(gt["y"]) - y_shift, 0)
        w = int(gt["w"])
        h = int(gt["h"])
        class_id = gt["class_id"]
        score = gt["class_confidence"]

        if class_id == 0:
            label = "GT_person"
            color = (0, 0, 255)  # red
        elif class_id == 2:
            label = "GT_car"
            color = (0, 255, 255)  # yellow
        else:
            continue
        x2, y2 = x + w, y + h
        cv2.rectangle(image, (x, y), (x2, y2), color, 2)
        cv2.putText(image, label, (x, max(0, y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def apply_nms(predictions, confidence_threshold=0.5, iou_threshold=0.4):
    """
    Apply Non-Maximum Suppression to event-based predictions.
    Each prediction should be a structured array with fields:
    ['x', 'y', 'w', 'h', 'confidence', ...]
    """
    boxes = []
    confidences = []

    for pred in predictions:
        conf = pred['class_confidence']
        if conf >= confidence_threshold:
            x = pred['x']
            y = pred['y']
            w = pred['w']
            h = pred['h']
            print(pred ['track_id'])
            boxes.append([int(x), int(y), int(w), int(h)])
            confidences.append(float(conf))

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, iou_threshold)

    if len(indices) == 0:
        return []

    indices = indices.flatten()
    return [predictions[i] for i in indices]

### Obtener las bboxes

In [ ]:

# Guardar Predicciones en imagenes
rgb_predictions_folder = "predictions/rgb" # NEED TO BE ADDED
event_predictions_path = "predictions"
sequences_folder = "rgb/test" # NEED TO BE ADDED
output_base_folder = "visual_results"
events_base_path = "--" # FOR LATER USE
gt_base_path = "/content/drive/MyDrive/DSEC/GT/test_object_detections/test"


sequences = [name for name in os.listdir(sequences_folder)
              if os.path.isdir(os.path.join(sequences_folder, name))]

sequences = ["interlaken_00_b"]

# remove thun_00_a from sequences
#sequences.remove("thun_02_a")
#sequences.remove("zurich_city_15_a")
#sequences.remove("zurich_city_14_a")
#sequences.remove("zurich_city_14_c")

print(sequences)

confidence_threshold_rgb = 0.5
confidence_threshold_event = 0.5


# For events and GT
event_y_shift = -50
gt_y_shift = 0

# Lista de secuencias
for sequence in sequences:
    print(f"Procesando secuencia: {sequence}")
    sequence_folder = os.path.join(sequences_folder, sequence)
    input_path = os.path.join(sequence_folder, "images/left/distorted")
    output_folder = os.path.join(output_base_folder, sequence)
    os.makedirs(output_folder, exist_ok=True)


    # Lista ordenada de imágenes
    image_files = sorted(
        [f for f in os.listdir(input_path) if f.endswith(".png")],
        key=lambda x: int(x.split('.')[0])
    )

    #-------------------- RGB DATA -----------------------------
    # Cargar RGB predicciones de la secuencia
    predictions_path = os.path.join(rgb_predictions_folder, f"{sequence}.npy")
    all_predictions = np.load(predictions_path, allow_pickle=True)

    # Agrupar por frame
    predictions_by_frame = {}
    for pred in all_predictions:
        frame_idx = pred["frame_idx"]
        if frame_idx not in predictions_by_frame:
            predictions_by_frame[frame_idx] = []
        predictions_by_frame[frame_idx].append(pred)

    print(len(predictions_by_frame))


    #-------------------- EVENT DATA -------------------------------
    # Cargar EVENT predicciones de la secuencia
    predictions_data = np.load(os.path.join(event_predictions_path, f"{sequence}.npz"), allow_pickle=True)
    timestamps_path = os.path.join(event_predictions_path, f"{sequence}_timestamps.npy")
    event_frame_timestamps = np.load(timestamps_path)

    # Extract prediction timestamps and detections
    predictions = predictions_data["predictions"]  # Shape: (N, ...)
    prediction_timestamps = predictions["t"]  # Assuming timestamp is the first column

    diffs = np.diff(prediction_timestamps)
    non_zero_arr = diffs[diffs != 0]

    # Initialize list to store detections per frame
    detections_per_frame = [[] for _ in range(len(event_frame_timestamps))]

    frame_idx = 0
    for i, ts in enumerate(prediction_timestamps):
        frame_idx = np.searchsorted(event_frame_timestamps, ts, side='right') - 1

        # Print debug info
        #print(f"\nPrediction {i}: ts = {ts}")
        #print(f"  → Candidate frame index: {frame_idx}")

        if 0 <= frame_idx < len(detections_per_frame):
            frame_start = event_frame_timestamps[frame_idx]
            frame_end = (
                event_frame_timestamps[frame_idx + 1]
                if frame_idx + 1 < len(event_frame_timestamps)
                else "END"
            )
            #print(f"  → Frame range: [{frame_start}, {frame_end})")

            if frame_end == "END" or ts < frame_end:
                #print(f"  ✓ Appending prediction {predictions[i]} to frame {frame_idx}")
                detections_per_frame[frame_idx].append(predictions[i])
            else:
                print(f"  ✗ Not in frame range — skipped.")

    print(len(detections_per_frame))


    # -------------------- GROUND TRUTH -----------------------
    # ACTUAL GT
    gt_path = os.path.join(gt_base_path, sequence,"object_detections","left","tracks.npy")
    gt_data = np.load(gt_path, allow_pickle=True)
    gt_timestamps = gt_data["t"]  # Assuming timestamp is the first column
    gt_per_frame = [[] for _ in range(len(event_frame_timestamps))]

    frame_idx = 0  # Start at the first frame

    for i, ts in enumerate(gt_timestamps):
        while (frame_idx + 1 < len(event_frame_timestamps)) and (ts >= event_frame_timestamps[frame_idx + 1]):
            frame_idx += 1

        # Clamp index to stay within range
        frame_idx = min(max(frame_idx, 0), len(gt_per_frame) - 1)

        gt_per_frame[frame_idx].append(gt_data[i])


    print(len(gt_per_frame))